### Practice for Python

This time you are going to write Python code yourself, performing data analysis and visualisation using Python and its libraries. You are going to read and plot Covid-19 data, comparing trends between different countries.

In [ ]:
# Import libraries
import os
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

### 1. Download the latest Covid-19 data from John Hopkins Univeristy repository.

To understand more about this dataset, please refer to: https://github.com/CSSEGISandData/COVID-19

In [ ]:
# The URL for Covid-19 statistics
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
f = '/time_series_covid19_confirmed_global.csv'

# Download csv files for confirmed case, death and recovered case data
print('Downloading csv files for confirmed case, death and recovered case data ...')
for f in ['time_series_covid19_confirmed_global.csv',
          'time_series_covid19_deaths_global.csv',
          'time_series_covid19_recovered_global.csv']:
    urllib.request.urlretrieve(URL + f, f)

# Download the csv file for country profiles
print('Downloading the csv file for country profiles ...')
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
f = 'UID_ISO_FIPS_LookUp_Table.csv'
urllib.request.urlretrieve(URL + f, f)

### 2. Read and show the confirmed cases for United Kingdom.

#### 2.1 Read the confirmed case csv file using Python pandas, print out its information.

#### 2.2 Select the UK Covid-19 data.

Note that there are several rows of UK data. We only analyse the whole UK data, not the regional data. For the whole UK data, 'Province/State' is NaN.

#### 2.3 Plot the cummulative confirmed cases in the UK by date.

#### 2.4 Plot the daily confirmed new cases in the UK by date.

#### 2.5 Smooth the daily case curve using a moving average window of 7 days.

#### 2.6 Check the curve on https://coronavirus.data.gov.uk/cases. Does your curve look similar?

#### 2.7 Compare the daily confirmed new case curves of several countries, including United Kingdom, France, Italy and US. You may also add countries that you are interested in. Smooth using a 7-day moving average window.

#### 2.8 Plot the curves of daily new cases (7-day moving averaged) per 100,000 population of these countries.
You can find the population information in the file UID_ISO_FIPS_LookUp_Table.csv.